In [1]:
import pandas as pd

In [4]:
df_us_2020 =pd.read_csv('../data/US_Data/2022_US_cohort.csv')

In [5]:
df_us_2020.head()

,birth_year,fridge_freezer,washing_machine,tumble_dryer,dishwasher,microwave,burglaries,car_crime,drunks,muggings,...,universal_credit,hosp_visits,gp_visits,depression,birth_month,academic_year,time,child_ages,age,weight
0,1969.0,1.0,1.0,1.0,1.0,1.0,-9.0,-9.0,-9.0,-9.0,...,0.0,2.0,1.0,-8.0,11.0,1969.0,2022,childless,52,-9.0
1,1970.0,1.0,1.0,1.0,1.0,1.0,-9.0,-9.0,-9.0,-9.0,...,-7.0,-7.0,-7.0,-7.0,1.0,1969.0,2022,childless,52,-9.0
2,1957.0,1.0,1.0,0.0,1.0,1.0,-9.0,-9.0,-9.0,-9.0,...,0.0,1.0,1.0,-8.0,8.0,1956.0,2022,childless,64,-9.0
3,1986.0,1.0,1.0,1.0,1.0,1.0,-9.0,-9.0,-9.0,-9.0,...,0.0,2.0,1.0,-8.0,6.0,1985.0,2022,3_6,36,-9.0
4,1977.0,1.0,1.0,1.0,1.0,1.0,-9.0,-9.0,-9.0,-9.0,...,0.0,2.0,2.0,-8.0,2.0,1976.0,2022,3_6,44,-9.0


In [9]:
print(df_us_2020.columns)

Index(['birth_year', 'fridge_freezer', 'washing_machine', 'tumble_dryer',
       'dishwasher', 'microwave', 'burglaries', 'car_crime', 'drunks',
       'muggings', 'racial_abuse', 'teenagers', 'vandalism', 'council_tax',
       'hh_netinc', 'region', 'heating', 'hidp', 'oecd_equiv', 'hh_int_m',
       'hh_int_y', 'job_duration_m', 'job_duration_y', 'emp_type', 'job_sec',
       'job_industry', 'job_occupation', 'labour_state_raw', 'ncigs', 'pidp',
       'education_state', 'newest_education_state', 'ethnicity', 'hh_rent',
       'loneliness', 'sex', 'SF_12', 'SF_12p', 'smoker', 'hh_mortgage',
       'alcohol_spending', 'nkids', 'nkids_ind_raw', 'nkids_ind_new',
       'ndrinks', 'yearly_electric', 'yearly_gas', 'yearly_gas_electric',
       'yearly_oil', 'yearly_other_fuel', 'has_electric', 'has_gas', 'has_oil',
       'has_other', 'has_none', 'gas_electric_combined', 'fruit_days',
       'fruit_per_day', 'veg_days', 'veg_per_day', 'hourly_rate',
       'gross_paypm', 'gross_pay_se', '